1. Obtener lista de antivirales con scrapeo de BD
2. Crear una columna de SMILES de esos antivirales
3. Pasar la columna de SMILES a la función de maic con selenium para obtener la afinidad
4. Crear una columna con esos valores

In [2]:
import pubchempy as pcp
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [3]:
def getSmilesByCID(cid):
    c = pcp.Compound.from_cid(cid)
    return c.canonical_smiles

def getSmilesByName(name):
    for compound in pcp.get_compounds(name, 'name'):
        return compound.canonical_smiles

In [4]:
page_number = 1

nombres = []
codigos = []

while page_number < 25:
    
    page_number += 1
    
    url = "https://www.drugbank.ca/unearth/q?approved=1&c=_score&ca=0&d=down&eu=0&page={}&query=antiviral&searcher=drugs&us=0".format(page_number)
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    hits = soup.find_all("h2",{"class":"hit-link"})
    
    for i in hits:
        #Crear la lista con sólo los nombres
        match = re.findall(r'DB\d{5}..(.*)</a></h2>', str(i))
        nombres.append(match)
        #Crear la lista con sólo los códigos
        db_code = re.findall(r'DB\d{5}', str(i))
        codigos.append(db_code)


In [5]:
print(len(nombres))
print(len(codigos))

81
81


In [6]:
df = pd.DataFrame(list(zip(nombres, codigos)))
df.head()

0          1
0   [Emtricitabine]  [DB00879]
1      [Zidovudine]  [DB00495]
2       [Darunavir]  [DB01264]
3      [Nevirapine]  [DB00238]
4  [Valganciclovir]  [DB01610]

In [7]:
#df.to_csv("output/antivirales.csv")

In [8]:
df.columns = ["nombre", "código"]

In [9]:
df['SMILES']=df.apply(lambda x: getSmilesByName(x.nombre), axis=1)

In [10]:
df.head()

nombre     código  \
0   [Emtricitabine]  [DB00879]   
1      [Zidovudine]  [DB00495]   
2       [Darunavir]  [DB01264]   
3      [Nevirapine]  [DB00238]   
4  [Valganciclovir]  [DB01610]   

                                              SMILES  
0                    C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F  
1         CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]  
2  CC(C)CN(CC(C(CC1=CC=CC=C1)NC(=O)OC2COC3C2CCO3)...  
3        CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4  
4       CC(C)C(C(=O)OCC(CO)OCN1C=NC2=C1N=C(NC2=O)N)N

In [11]:
#df.to_csv("output/antiviralesconsmiles.csv")

In [12]:
## FASTAS

proteinas = pd.read_csv("output/55_FASTA.csv")
proteinas.head()

id_cadena                                              FASTA
0    6YB7_A  SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
1    6W75_D  SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
2    6W75_C  SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...
3    6W75_B  SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
4    6W75_A  SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...

In [13]:
data = pd.DataFrame()

for drug_index in df.transpose():
    for protein_index in proteinas.transpose():
        data = data.append(pd.DataFrame({'Nombre':df['nombre'][drug_index],
                                          'SMILES':df['SMILES'][drug_index],
                                          'id_cadena':proteinas['id_cadena'][protein_index],
                                          'FASTA':proteinas['FASTA'][protein_index]}))
        
    
        

In [16]:
data.to_csv("output/data.csv")
data.head()

Nombre                           SMILES id_cadena  \
0  Emtricitabine  C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F    6YB7_A   
0  Emtricitabine  C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F    6W75_D   
0  Emtricitabine  C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F    6W75_C   
0  Emtricitabine  C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F    6W75_B   
0  Emtricitabine  C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F    6W75_A   

                                               FASTA  
0  SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...  
0  SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...  
0  SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...  
0  SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...  
0  SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...

In [17]:
lst = []

for i in range(len(data)):
    dct = {
        "protein_ID": data.iloc[i]["id_cadena"],
        "FASTA": data.iloc[i]["FASTA"],
        "small_mol_name": data.iloc[i]["Nombre"][0],
        "SMILES": data.iloc[i]["SMILES"]}
    lst.append(dct)


In [79]:
'''
with open('output/data.txt', 'w') as file:
    for listitem in lst:
        file.write('%s\n' % listitem)
'''

In [1]:
#print(lst)

## 2 protein Sample: 

6W41-C: Chain C, Spike glycoprotein receptor binding domain

6YB7-A: Chain A, SARS-CoV-2 main protease


In [34]:
sample = data[(data["id_cadena"] == "6W41_C") | (data["id_cadena"] == "6YB7_A")]

sample.shape

(162, 4)

In [35]:
lst = []

for i in range(len(sample)):
    dct = {
        "protein_ID": sample.iloc[i]["id_cadena"],
        "FASTA": sample.iloc[i]["FASTA"],
        "small_mol_name": sample.iloc[i]["Nombre"][0],
        "SMILES": sample.iloc[i]["SMILES"]}
    lst.append(dct)

In [38]:
print(lst)

[{'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ', 'small_mol_name': 'E', 'SMILES': 'C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F'}, {'protein_ID': '6W41_C', 'FASTA': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFSGHHHHHH', 'small_mol_name': 'E', 'SMILES': 'C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F'}, {'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVI